# Lecture 11: Brand new DSL world
- GADTs
- Existential type
- Rank N types
- DSL

## GADT

GADT - как ADT, только мы явно задаем типы конструкторов. 
Т.е. до этого у нас было
 - Branch (Tree a) a (Tree a)  что раскрывалось в Tree a -> a -> Tree a -> Tree a

А теперь станет 
 - Branch :: (Tree Int) -> Int -> (Tree Int) -> (Tree Int)

Это позволило типизировать конструкторы точнее, нежели чем опираясь на заданный заренее тип

Сейчас рассмотрим пример и станет понятней.



## DSL для выражений

DSL - язык заточенный под опр задачу

Хотим написать DSL для чисел и булов.

Но сталкнемся с некоторй глиной

Хотим:

```
34 + 45 + 26

23 + 45 > 107

(23 + 304 > 107) && (11 > 8 + 5)
```

Определяем: 
```
data ArithExpr =
    AENum Int
  | AEPlus ArithExpr ArithExpr
  | AEAnd ArithExpr ArithExpr
  | AEGt ArithExpr ArithExpr

-- (23 + 12) > 170 && (35 > 47)
myExpr =
  ((AENum 23 `AEPlus` AENum 12) `AEGt` AENum 170)
  `AEAnd` (AENum 35 `AEGt` AENum 47)
```

Однако мы спокойно можем сделать AND на числах - not cool. Завсем не уследишь сам. Соотв интропретатор будет таким:
```
interpret :: ArithExpr -> Either String (Either Int Bool)
interpret (AENum n) = pure $ Left n
interpret (AEAnd a b) = do
  a' <- interpretBool a
  b' <- interpretBool b
  pure $ Right (a' && b')

interpretBool :: ArithExpr -> Either String Bool
interpretBool x =
  interpret x >>= either (\x -> Left $ "Unexpected " <> show x) pure
```
Что будет кидать нам ошибко, что ок. Хотя лучше бы запретить такие действия на уровне компилятора

Посмотрим сейчас на тип операций:
```
ghci> :t AENum
AENum  :: Int -> ArithExpr

ghci> :t AEPlus
AEPlus :: ArithExpr -> ArithExpr -> ArithExpr
```

Заметим, что нам хотелось бы иметь из от Int и Bool соотв.

\*GADT\*
```
data ArithExpr a where
  AENum  :: Int -> ArithExpr Int
  AEPlus :: ArithExpr Int -> ArithExpr Int -> ArithExpr Int
  AEAnd  :: ArithExpr Bool -> ArithExpr Bool -> ArithExpr Bool
  AEGt   :: ArithExpr Int -> ArithExpr Int -> ArithExpr Bool
```
Теперь мы явно указываем, какой тип возвращает выражение. И создать невалидный объект у нас просто нет возможности

И интропретатор теперь выглядит так:

```
interpret :: ArithExpr a -> a
interpret (AENum n) = n
interpret (AEPlus a b) = interpret a + interpret b
interpret (AEAnd a b) = interpret a && interpret b
interpret (AEGt a b) = interpret a > interpret b
```

Используются конструкторы точно так же, как и обычные.




## Exestential types

 Экзистенциальные типы это такая штука, которая помогает убрать информацию о типе, так как у нас для функций всегда, хоть и неявно, стоит forall, а мы хотим что-то типа exist


### Парсер DSL

Попробуем в пасрер. Пока очень простой

```
parse
  :: String -> Maybe (ArithExpr a)
parse "1" = Just (AENum 1)
parse _ = Nothing
```
И получим: 
```
ghci> :l DSL.hs
error:
    • Couldn't match type ‘a’ with ‘Int’
      Expected type: Maybe (ArithExpr a)
        Actual type: Maybe (ArithExpr Int)
```
Тут мы буквально сказали хаскелю: для любого типа а мы спарсим выражение. Но нам хочется скорее сказать: существует такой тип, для которого мы это спарсим.

Тогда создадим вспом класс:

```
data SomeAE where
  SomeAE :: Show a => ArithExpr a -> SomeAE

parse :: String -> Maybe SomeAE
parse "1" = Just (SomeAE $ AENum 1)
parse "1+2" = Just $ SomeAE $
  AENum 1 `AEPlus` AENum 2
parse _ = Nothing

interpretShow :: SomeAE -> String
interpretShow (SomeAE expr) =
    show (interpret expr)
```
Альтернативно можно сделать так:
```
{-# LANGUAGE ExistentialQuantification #-}

data SomeAE =
  forall a. Show a => SomeAE (ArithExpr a)
```

SomeAE хранит в себе объект ArithExpr a для какого-то одного а. Теперь смысл сошелся.

Как теперь создать функцию, которая будет парсить только один конкретный тип? Для этого надо вызвать parse и проверить, что тип выражения, который он распарсил, совпал с нужным. Делается это следующим образом:

```
-- | The class 'Typeable' allows
-- a concrete representation of
-- a type to be calculated.
class Typeable (a :: k)

-- | Propositional equality.
-- If @a :~: b@ is inhabited by some
-- terminating value, then the type @a@
-- is the same as the type @b@.
data a :~: b where
  Refl :: a :~: a

-- | Extract a witness of equality
-- of two types
eqT
  :: forall a b. (Typeable a, Typeable b)
  => Maybe (a :~: b)
```
```
{-# LANGUAGE TypeApplications #-}
{-# LANGUAGE ScopedTypeVariables #-}

parseInt
  :: String -> Maybe (ArithExpr Int)
parseInt s = parse s >>=
  \(SomeAE (expr :: ArithExpr t)) ->
    do
      Refl <- eqT @t @Int
      pure expr

ghci> let aE = parseInt "1+2"
ghci> aE
Just 1 + 2
ghci> (^3) . interpret <$> aE
Just 27
```

Мы добавили для типа а в SomeAE инстанс Typeable, который говорит, что мы можем посчитать тип этого объекта. 

Далее определяем тип a :~: b, у которого существует единственный конструктор Refl, подразумевающий, что а и b это один и тот же тип. 

Функция eqT же параметризуется двумя типами и возвращает Just Refl, если они совпали, и Nothing иначе.

В итоге в функции parseInt мы производим паттерн-матчинг, включающий в себя тип t, а после вызываем функцию eqT на этом типе и типе Int. Так как eqT возвращает Maybe, то мы можем ее использовать внутри нашей do нотации. Итого, если паттерн-матчинг по Refl сойдется, мы вернем pure expr, иначе Nothing будет прокинут до конца вычислений.

## Rank N types
Позволяет принимать полиморфные функции. И чем глубже она тем выше Rank

Рассмотрим forall

```
length :: [a] -> Int
---  раскрываеться в:
length :: forall a . [a] -> Int

applyToTuple :: ([a] -> Int) -> ([b], [c]) -> (Int, Int)
applyToTuple f x y = (f x, f y)

-- Couldn't match type 'b' with 'a' ...
-- Couldn't match type 'c' with 'a' ...
```
не скомпилилось...

Почему? A потому что для любых типов a, b, c

Тут вступает понятие Rank, нам надо как бы сказать что мы передаем полиморфную функцию и нам не нужно форолить а вместе с c b

```
{-# LANGUAGE RankNTypes #-}

applyToTuple :: (forall a. [a] -> Int) -> ([b], [c]) -> (Int, Int)
applyToTuple f (x, y) = (f x, f y)

applyToTuple length ("hello", [1,2,3])

```
Глубина вложенности квантора forall называется ранком:

примеры ранков

```
length :: forall a . [a] -> Int            -- Rank-1-Types
id     :: forall a . a -> a                -- Rank-1-Types
show   :: forall a . Show a => a -> String -- Rank-1-Types

Rank 0: Int
Rank 1: forall a . a -> Int
Rank 2: (forall a . a -> Int) -> Int           -- could be enabled by Rank2Types
Rank 3: ((forall a . a -> Int) -> Int) -> Int

Int -> Int                                   -- rank 0
forall a . a -> a
forall a . a -> a                            -- rank 1
The rank describes the depth at which universal quantifiers appear in a contravariant position, i.e. to the left of a function arrow.
(forall a . a -> a) -> Int
(forall a . a -> a) -> Int                   -- rank 2
Int -> (forall a . a -> a) 
forall a . Int -> a -> a                     -- rank 1
forall a b . a -> b -> a
forall a b . a -> b -> a                     -- rank 1 
forall a . a -> (forall b . b -> a)          -- rank 1

(a -> a) -> (forall b . b -> b) -> (c -> c)  -- rank 2
```

Но не так хаскель не может так просто понять что нужно юзать Rank 2. Он всегда считает в Rank 1

Если ранк больше 2, то задача вывода типа становится неразрешимой. Обычно никогда не нужны ранки выше 2.
```
applyTwo :: ([Int], [Bool])
applyTwo = let call f = (f [2, 1, 3], f [True, False]) in call reverse  -- (*)
```
Хотелось бы
```
-- and we want
gchi> applyTwo
([3,1,2], [False,True])
```
А получим
```
-- but we get:
(*):29:
    No instance for (Num Bool) arising from the literal ‘2’
    In the expression: 2
    In the first argument of ‘f’, namely ‘[2, 1, 3]’
    In the expression: f [2, 1, 3]

(*):15:
    Couldn't match type ‘Int’ with ‘Bool’
    Expected type: ([Int], [Bool])
      Actual type: ([Int], [Int])
    In the expression: call reverse
    In the expression: let call f = (f ..., f ...) in call reverse

(*):20:
    Couldn't match type ‘Bool’ with ‘Int’
    Expected type: [Bool] -> [Int]
      Actual type: [Int] -> [Int]
    In the first argument of ‘call’, namely ‘reverse’
    In the expression: call reverse

```
Она не скомпилится. Компилятор подумает: мы применили f к списку интов. Значит это функция из списка интов. Потом, во время применения к списку булов, она скажет “типы не сошлись”. Поэтому надо явно указать слева от равно тип функции f


### ST revisited

ST - нужна для последовательных pure вычеслений

```
runST :: forall α. (forall s. ST s α) -> α

newSTRef :: forall α s. α -> ST s (STRef s α)
readSTRef :: forall α s. STRef s α -> ST s α
writeSTRef :: forall α s. STRef s α -> α -> ST s ()
```

```
mutableActionWrong :: Int
mutableActionWrong = runST $ do
    var <- newSTRef 5
    let newVar = changeVar var
    readSTRef newVar
```

```
mutableAction :: Int
mutableAction = runST $ do
    var <- newSTRef 5
    changeVar var
    readSTRef newVar
```

```
changeVarWrong :: STRef s Int -> STRef s Int
changeVarWrong var =
  let _ = runST $ writeSTRef var 10
      _ = runST $ writeSTRef var 42
   in var
```

```
changeVar :: STRef s Int -> ST s ()
changeVar var = runST $ do
  writeSTRef var 10
  writeSTRef var 42

```

Если делать не правильно, то будем получать:
```
Couldn't match type ‘s’ with ‘s1’
      ‘s’ is a rigid type variable bound by
          the type signature for changeVarWrong :: STRef s a -> STRef s a
      ‘s1’ is a rigid type variable bound by
           a type expected by the context: ST s1 () at (*)
    Expected type: STRef s1 a
      Actual type: STRef s a
```

Как бы внутренний тип не позволяет неправильно исп ST, т.к. не приводиться друг к другу, а достать мы его оттуда не можем

### forall в data type

```
data Ctx = Ctx { modulus :: Int }

newtype Action a = Action
  { runAction
      :: forall m .
        (MonadReader Ctx m, MonadCatch m)
      => m a }
```
```
expOrDefault
  :: Int -> Int -> Int -> Action Int
expOrDefault base pow def = Action $ do
  m <- asks modulus
  let r = base ^ pow
  (r `seq` pure (r `mod` m))
    `catchAny` \_ -> pure def

runPrint = runAction >=> print

main :: IO ()
main = flip runReaderT (Ctx 17) $ do
  runPrint $ expOrDefault 2 5  (-100)
  runPrint $ expOrDefault 2 (-1) (-100)
```
```
data Action a = 
 forall m .
  (MonadReader Ctx m, MonadCatch m)
 => Action (m a)
 ```
 ```
 data Action a where 
  Action :: forall m .
   (MonadReader Ctx m, MonadCatch m)
   => m a  
   -> Action a
```

### напоминание про ScopedTypeVariables

```
calc :: Num a => a -> a -> a
calc a b = a + f b
  where
    f :: a -> a
    f = (+ 10)
```
получим
```
Forall1.hs:40:10: error:
    • Could not deduce (Num a1)
      arising from a use of ‘+’
      from the context: Num a
```

С этим все ок:
```
calc2 :: Num a => a -> a -> a
calc2 a b = a + f b
  where
    f :: forall a. Num a => a -> a
    f = (+ 10)
```
Тоже ок:

```
{-# LANGUAGE ScopedTypeVariables #-}

calc3 :: forall a. Num a => a -> a -> a
calc3 a b = a + f b
  where
    f :: a -> a
    f = (+ 10)
```

```
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE TypeApplications #-}

calc3 :: forall a. Num a => a -> a -> a
calc3 a b = a + f @a b

f :: Num a => a -> a
f = (+ 10)
```

## tagless DSL

давайте тепреь попробуем сделать по другому. Зададим DSL не как data, а как class

```
class ArithExpr expr where
  aeNum  :: Int -> expr Int
  aePlus :: expr Int -> expr Int -> expr Int
  aeAnd  :: expr Bool -> expr Bool -> expr Bool
  aeGt   :: expr Int -> expr Int -> expr Bool


```
```
myExpr :: ArithExpr expr => expr Bool
myExpr = ((aeNum 23 `aePlus` aeNum 12) `aeGt` aeNum 170)
           `aeAnd` (aeNum 35 `aeGt` aeNum 47)
```
пока это что то что мы определили, но юзать не можем

```
newtype ToS a = ToS {toString :: String}
  deriving (Show, Semigroup)

castTS :: ToS a -> ToS b
castTS (ToS s) = ToS s

instance ArithExpr ToS where
  aeNum = ToS . show
  aePlus a b = a <> (ToS " + ") <> b
  aeAnd a b = a <> (ToS " && ") <> b
  aeGt a b =
    castTS a <> (ToS " > ") <> castTS b

```

```
newtype Interpret a =
  Interpret { interpret :: a }

instance ArithExpr Interpret where
  aeNum = Interpret
  aePlus a b = Interpret $
    interpret a + interpret b
  aeAnd a b = Interpret $
    interpret a && interpret b
  aeGt a b = Interpret $
    interpret a > interpret b

```

```
myExpr :: ArithExpr expr => expr Bool
myExpr = ((aeNum 23 `aePlus` aeNum 12)
           `aeGt` aeNum 170) `aeAnd`
           (aeNum 35 `aeGt` aeNum 47)

ghci> toString myExpr
"23 + 12 > 170 && 35 > 47"

ghci> interpret myExpr
False
```


За счет типизации все работает! Ура

### Why care about DSLs?
With power of type system, extensive syntax and support of both strict and lazy, pure and effectful computations,
Haskell is great for embedded DSLs.